In [51]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from numpy.linalg import norm

In [55]:
from google.colab import drive

# Monter Google Drive
drive.mount('/content/drive')
!ls '/content/drive/My Drive/Colab Notebooks/Jedha/Projet/data/'
path = "/content/drive/My Drive/Colab Notebooks/Jedha/Projet/data/roberta_results.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
data_avec_labels.csv  model_roberta_results.csv  model_roberta_results.csv.csv	roberta_results.csv


In [54]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [64]:
df = pd.read_csv(path, encoding='latin-1')

In [57]:
df.columns

Index(['Unnamed: 0', 'reviewer_id', 'store_address', 'latitude', 'longitude',
       'review_time', 'review', 'rating', 'review_date', 'City', 'State',
       'ZipCode', 'rating_int', 'clean_reviews', 'actual_sentiment',
       'pred_sentiment', 'RoBERTa_score'],
      dtype='object')

In [58]:
df.head()

,Unnamed: 0,reviewer_id,store_address,latitude,longitude,review_time,review,rating,review_date,City,State,ZipCode,rating_int,clean_reviews,actual_sentiment,pred_sentiment,RoBERTa_score
0,0,1,13749 US-183 Hwy,30.460718,-97.792874,3 months ago,"Why does it look like someone spit on my food?\nI had a normal transaction, everyone was chill ...",1 star,2025-01-08,Austin,TX,78750,1,Why does it look like someone spit on my food I had a normal transaction everyone was chill and ...,negative,negative,0.917278
1,1,2,13749 US-183 Hwy,30.460718,-97.792874,5 days ago,It'd McDonalds. It is what it is as far as the food and atmosphere go. The staff here does make ...,4 stars,2025-04-03,Austin,TX,78750,4,Itd McDonalds It is what it is as far as the food and atmosphere go The staff here does make a d...,positive,positive,0.972301
2,2,3,13749 US-183 Hwy,30.460718,-97.792874,5 days ago,Made a mobile order got to the speaker and checked it in.\nLine was not moving so I had to leave...,1 star,2025-04-03,Austin,TX,78750,1,Made a mobile order got to the speaker and checked it in Line was not moving so I had to leave o...,negative,negative,0.661857
3,3,4,13749 US-183 Hwy,30.460718,-97.792874,a month ago,My mc. Crispy chicken sandwich was .................................... customer service was qui...,5 stars,2025-03-08,Austin,TX,78750,5,My mc Crispy chicken sandwich was customer service was quick and p,positive,positive,0.940411
4,4,5,13749 US-183 Hwy,30.460718,-97.792874,2 months ago,"I repeat my order 3 times in the drive thru, and she still manage to mess it up , it was suppose...",1 star,2025-02-08,Austin,TX,78750,1,I repeat my order times in the drive thru and she still manage to mess it up it was suppose to b...,negative,negative,0.913123


In [59]:
#pour vérifier que la limite de token du model est suffisante
print("longeure moyenne :" , df['review'].apply(len).mean())
print("longeure mediane :" , df['review'].apply(len).median())
print("longeure max :" , df['review'].apply(len).max())
print("longeure min :" , df['review'].apply(len).min())

longeure moyenne : 142.5908215661104
longeure mediane : 83.0
longeure max : 2553
longeure min : 1


# Embedding des labels

In [63]:
labels = [
    'hygiene', 'food quality', 'food', 'staff', 'something is missing',
    'location', 'speed of service', 'drive-thru', 'temperature of the food',
    'atmosphere', 'customer service', "temperature" , "price", "speed", "quality", "courtesy",

]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') #embeding à 384 dimentions, max token = 256
labels_embedded = model.encode(labels)

# test du l'embedin et dot product sur un seul restaurant

In [ ]:
df["store_address"][0]

'13749 US-183 Hwy, Austin, TX 78750, United States'

In [ ]:
#selection du premier restaurant
df_1restaurant = df[df["store_address"]=="13749 US-183 Hwy, Austin, TX 78750, United States"]
df_1restaurant.shape

(578, 16)

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') #embeding à 384 dimentions, max token = 256
reviews_embedded = model.encode(df_1restaurant["review"])
df_1restaurant['review_embedded'] = reviews_embedded.tolist()

<ipython-input-194-6e20b6632105>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1restaurant['review_embedded'] = reviews_embedded.tolist()


In [ ]:
df_1restaurant["review_embedded"].shape

(578,)

In [ ]:
df_1restaurant['review_embedded'].head()

,review_embedded
0,"[0.022007549181580544, -0.048939093947410583, 0.03766841068863869, -0.0076949819922447205, 0.025..."
1,"[0.03945782035589218, -0.015777520835399628, 0.07758671045303345, 0.013737925328314304, 0.014259..."
2,"[-0.05103641003370285, 0.04763896018266678, 0.05201364681124687, -0.04680230841040611, 0.0035369..."
3,"[-0.04021293297410011, -0.024235272780060768, 0.05131354555487633, -0.009905711747705936, -0.025..."
4,"[0.04103447124361992, 0.006413918454200029, 0.12267220765352249, 0.021540647372603416, -0.102571..."


In [ ]:
# Calculer le produit scalaire normalisé pour chaque label et trouver le label avec le score le plus élevé
top_labels = []
for review in reviews_embedded:
    label_scores = [
        (label, np.dot(review, labels_embedded[i]) / (norm(review) * norm(labels_embedded[i])))
        for i, label in enumerate(labels)
    ]
    # Trier les labels par score décroissant
    sorted_labels = sorted(label_scores, key=lambda x: x[1], reverse=True)
    # Sélectionner les cinq premiers labels
    top_labels.append(sorted_labels[:5])

# Ajouter des colonnes pour les cinq labels ayant les scores les plus élevés
for i in range(5):
    df_1restaurant[f'top_{i+1}_label'] = [labels[i] for labels in top_labels]

# Augmenter la limite de caractères affichés par cellule
pd.set_option('display.max_colwidth', 100)

df_1restaurant[["review",\
                'top_1_label', 'top_2_label','top_3_label', 'top_4_label', 'top_5_label']].head(10)



<ipython-input-197-d9e6b03c5bf1>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1restaurant[f'top_{i+1}_label'] = [labels[i] for labels in top_labels]
<ipython-input-197-d9e6b03c5bf1>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1restaurant[f'top_{i+1}_label'] = [labels[i] for labels in top_labels]
<ipython-input-197-d9e6b03c5bf1>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

,review,top_1_label,top_2_label,top_3_label,top_4_label,top_5_label
0,"Why does it look like someone spit on my food?\nI had a normal transaction, everyone was chill ...","(hygiene, 0.28746822)","(food, 0.27422136)","(temperature of the food, 0.25352862)","(food quality, 0.19775568)","(drive-thru, 0.17217408)"
1,It'd McDonalds. It is what it is as far as the food and atmosphere go. The staff here does make ...,"(staff, 0.3886896)","(food quality, 0.36147067)","(customer service, 0.30244836)","(food, 0.2999506)","(temperature of the food, 0.226313)"
2,Made a mobile order got to the speaker and checked it in.\nLine was not moving so I had to leave...,"(customer service, 0.3827942)","(staff, 0.14755319)","(drive-thru, 0.14341448)","(something is missing, 0.14244273)","(location, 0.14115314)"
3,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï...,"(food quality, 0.42417273)","(temperature of the food, 0.35612532)","(food, 0.33923125)","(customer service, 0.296566)","(speed of service, 0.19602737)"
4,"I repeat my order 3 times in the drive thru, and she still manage to mess it up , it was suppose...","(drive-thru, 0.40096334)","(food, 0.26292616)","(food quality, 0.22811927)","(customer service, 0.1991002)","(temperature of the food, 0.18270876)"
5,I work for door dash and they locked us all out to wait in a long line for no reason at 10. I tr...,"(customer service, 0.2676896)","(drive-thru, 0.1942678)","(staff, 0.16171806)","(location, 0.14941879)","(speed of service, 0.13265584)"
6,"If I could give this location a zero on customer service, I would. Pulled into drive through a...","(customer service, 0.3626177)","(drive-thru, 0.24972373)","(food quality, 0.18007174)","(staff, 0.17227705)","(speed of service, 0.17197515)"
7,Came in and ordered a Large coffee w/no ice. They handed me a cup that was less than half way fu...,"(drive-thru, 0.1847711)","(customer service, 0.1640311)","(temperature of the food, 0.14997266)","(food quality, 0.11158571)","(staff, 0.10174852)"
8,Went thru drive thru. Ordered. Getting home noticed my 10 piece nuggets where missing. Tried cal...,"(something is missing, 0.31157857)","(food, 0.29818207)","(food quality, 0.28817856)","(drive-thru, 0.28361377)","(temperature of the food, 0.26132107)"
9,"I'm not really a huge fan of fast food, but I have 2 teenage daughter's who enjoy McDs. The staf...","(food quality, 0.38792324)","(customer service, 0.29408354)","(drive-thru, 0.26169395)","(speed of service, 0.25601938)","(food, 0.22585614)"


In [ ]:
df_1restaurant.columns

Index(['Unnamed: 0', 'reviewer_id', 'pred_sentiment', 'RoBERTa_score',
       'clean_reviews', 'rating', 'actual_sentiment', 'correct_prediction',
       'latitude ', 'longitude', 'review_date', 'City', 'State', 'review',
       'store_address', 'review_time', 'review_embedded', 'top_1_label',
       'top_2_label', 'top_3_label', 'top_4_label', 'top_5_label'],
      dtype='object')

# Embedding et dot product sur tout le dataset

In [66]:
from tqdm.notebook import tqdm

# Remplacer les valeurs manquantes par une chaîne vide
df['clean_reviews'] = df['clean_reviews'].fillna('')

# Charger le modèle SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Encoder les avis avec une barre de progression
reviews_embedded = []
for review in tqdm(df["clean_reviews"], desc="Encoding reviews"):
    reviews_embedded.append(model.encode(review))

# Ajouter les embeddings au DataFrame
df['review_embedded'] = reviews_embedded

Encoding reviews:   0%|          | 0/21812 [00:00<?, ?it/s]

In [67]:
# Calculer le produit scalaire normalisé pour chaque label et trouver le label avec le score le plus élevé
top_labels = []
for review in reviews_embedded:
    label_scores = [
        (label, np.dot(review, labels_embedded[i]) / (norm(review) * norm(labels_embedded[i])))
        for i, label in enumerate(labels)
    ]
    # Trier les labels par score décroissant
    sorted_labels = sorted(label_scores, key=lambda x: x[1], reverse=True)
    # Sélectionner les cinq premiers labels
    top_labels.append(sorted_labels[:5])

# Ajouter des colonnes pour les cinq labels ayant les scores les plus élevés
for i in range(5):
    df[f'top_{i+1}_label'] = [labels[i] for labels in top_labels]

# Augmenter la limite de caractères affichés par cellule
pd.set_option('display.max_colwidth', 100)

df[["review",\
   'top_1_label', 'top_2_label','top_3_label', 'top_4_label', 'top_5_label']].head(10)



,review,top_1_label,top_2_label,top_3_label,top_4_label,top_5_label
0,"Why does it look like someone spit on my food?\nI had a normal transaction, everyone was chill ...","(hygiene, 0.2955256)","(food, 0.29029495)","(temperature of the food, 0.253002)","(food quality, 0.19989088)","(courtesy, 0.19330013)"
1,It'd McDonalds. It is what it is as far as the food and atmosphere go. The staff here does make ...,"(food quality, 0.44993627)","(staff, 0.28324518)","(food, 0.26693666)","(customer service, 0.26407588)","(temperature of the food, 0.23549603)"
2,Made a mobile order got to the speaker and checked it in.\nLine was not moving so I had to leave...,"(customer service, 0.37688273)","(courtesy, 0.15730277)","(location, 0.15451363)","(drive-thru, 0.14108482)","(speed of service, 0.1278376)"
3,My mc. Crispy chicken sandwich was .................................... customer service was qui...,"(customer service, 0.46452987)","(speed of service, 0.3742587)","(food quality, 0.3491085)","(food, 0.29163992)","(quality, 0.26369035)"
4,"I repeat my order 3 times in the drive thru, and she still manage to mess it up , it was suppose...","(drive-thru, 0.3945921)","(courtesy, 0.26226512)","(food, 0.2576281)","(food quality, 0.23122598)","(customer service, 0.20002954)"
5,I work for door dash and they locked us all out to wait in a long line for no reason at 10. I tr...,"(customer service, 0.2917828)","(drive-thru, 0.20457889)","(location, 0.17262074)","(staff, 0.15753199)","(speed of service, 0.12911645)"
6,"If I could give this location a zero on customer service, I would. Pulled into drive through a...","(customer service, 0.4158214)","(drive-thru, 0.2791837)","(courtesy, 0.26778528)","(speed of service, 0.22346093)","(location, 0.20941906)"
7,Came in and ordered a Large coffee w/no ice. They handed me a cup that was less than half way fu...,"(customer service, 0.22562192)","(drive-thru, 0.20950341)","(courtesy, 0.14556743)","(staff, 0.142205)","(price, 0.13425298)"
8,Went thru drive thru. Ordered. Getting home noticed my 10 piece nuggets where missing. Tried cal...,"(drive-thru, 0.32190916)","(food, 0.3210567)","(food quality, 0.3103828)","(something is missing, 0.31007636)","(temperature of the food, 0.25788894)"
9,"I'm not really a huge fan of fast food, but I have 2 teenage daughter's who enjoy McDs. The staf...","(food quality, 0.36859584)","(customer service, 0.32133225)","(drive-thru, 0.2600027)","(speed of service, 0.25963047)","(staff, 0.2279775)"


In [68]:
df.columns

Index(['Unnamed: 0', 'reviewer_id', 'store_address', 'latitude', 'longitude',
       'review_time', 'review', 'rating', 'review_date', 'City', 'State',
       'ZipCode', 'rating_int', 'clean_reviews', 'actual_sentiment',
       'pred_sentiment', 'RoBERTa_score', 'review_embedded', 'top_1_label',
       'top_2_label', 'top_3_label', 'top_4_label', 'top_5_label'],
      dtype='object')

In [69]:
df=df[['reviewer_id',
       'pred_sentiment', 'RoBERTa_score',
       'review',
       'top_1_label','top_2_label', 'top_3_label', 'top_4_label', 'top_5_label',
       'review_time', 'review_date',
       'store_address', 'latitude', 'longitude','City', 'State',
       'rating_int', 'actual_sentiment' ]]
df.head()

,reviewer_id,pred_sentiment,RoBERTa_score,review,top_1_label,top_2_label,top_3_label,top_4_label,top_5_label,review_time,review_date,store_address,latitude,longitude,City,State,rating_int,actual_sentiment
0,1,negative,0.917278,"Why does it look like someone spit on my food?\nI had a normal transaction, everyone was chill ...","(hygiene, 0.2955256)","(food, 0.29029495)","(temperature of the food, 0.253002)","(food quality, 0.19989088)","(courtesy, 0.19330013)",3 months ago,2025-01-08,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,1,negative
1,2,positive,0.972301,It'd McDonalds. It is what it is as far as the food and atmosphere go. The staff here does make ...,"(food quality, 0.44993627)","(staff, 0.28324518)","(food, 0.26693666)","(customer service, 0.26407588)","(temperature of the food, 0.23549603)",5 days ago,2025-04-03,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,4,positive
2,3,negative,0.661857,Made a mobile order got to the speaker and checked it in.\nLine was not moving so I had to leave...,"(customer service, 0.37688273)","(courtesy, 0.15730277)","(location, 0.15451363)","(drive-thru, 0.14108482)","(speed of service, 0.1278376)",5 days ago,2025-04-03,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,1,negative
3,4,positive,0.940411,My mc. Crispy chicken sandwich was .................................... customer service was qui...,"(customer service, 0.46452987)","(speed of service, 0.3742587)","(food quality, 0.3491085)","(food, 0.29163992)","(quality, 0.26369035)",a month ago,2025-03-08,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,5,positive
4,5,negative,0.913123,"I repeat my order 3 times in the drive thru, and she still manage to mess it up , it was suppose...","(drive-thru, 0.3945921)","(courtesy, 0.26226512)","(food, 0.2576281)","(food quality, 0.23122598)","(customer service, 0.20002954)",2 months ago,2025-02-08,13749 US-183 Hwy,30.460718,-97.792874,Austin,TX,1,negative


In [71]:
df.shape

(21812, 18)

In [72]:
df.to_csv("/content/drive/My Drive/Colab Notebooks/Jedha/Projet/data/data_avec_labels.csv", index=False)

# process pour intégrer les colonnes labels (finalement fait dans le fichier .py)

In [ ]:
# Initialiser les nouvelles colonnes avec des zéros
for label in labels:
    df[label] = 0

# Mettre à jour les colonnes en fonction des labels et des scores
for index, row in df.iterrows():
    for col in ['top_1_label', 'top_2_label', 'top_3_label', 'top_4_label', 'top_5_label']:
        label, score = row[col]
        if score > 0.25 and label in labels:
            df.at[index, label] = 1

<ipython-input-36-2e04e464cdf1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[label] = 0
<ipython-input-36-2e04e464cdf1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[label] = 0
<ipython-input-36-2e04e464cdf1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [ ]:
df.columns

Index(['reviewer_id', 'pred_sentiment', 'RoBERTa_score', 'clean_reviews',
       'review', 'top_1_label', 'top_2_label', 'top_3_label', 'top_4_label',
       'top_5_label', 'rating', 'actual_sentiment', 'latitude ', 'longitude',
       'City', 'State', 'store_address', 'review_time', 'review_date',
       'hygiene', 'food quality', 'food', 'staff', 'something is missing',
       'location', 'speed of service', 'drive-thru', 'temperature of the food',
       'atmosphere', 'customer service', 'temperature', 'price', 'speed',
       'quality', 'courtesy'],
      dtype='object')

In [ ]:
df=df[['reviewer_id', 'pred_sentiment', 'RoBERTa_score', 'clean_reviews',
       'review', 'top_1_label', 'top_2_label', 'top_3_label', 'top_4_label',
       'top_5_label',  'hygiene', 'food quality', 'food',
       'staff', 'something is missing', 'location', 'speed of service',
       'drive-thru', 'temperature of the food', 'atmosphere',
       'customer service', 'temperature', 'price', 'speed', 'quality',
       'courtesy','rating', 'actual_sentiment',
       'latitude ', 'longitude', 'City', 'State', 'store_address',
       'review_time', 'review_date' ]]

In [ ]:
df[['clean_reviews', 'top_1_label', 'top_2_label', 'top_3_label', 'top_4_label',
       'top_5_label',  'hygiene', 'food quality', 'food',
       'staff', 'something is missing', 'location', 'speed of service',
       'drive-thru', 'temperature of the food', 'atmosphere',
       'customer service', 'temperature', 'price', 'speed', 'quality',
       'courtesy']].head()

,clean_reviews,top_1_label,top_2_label,top_3_label,top_4_label,top_5_label,hygiene,food quality,food,staff,...,speed of service,drive-thru,temperature of the food,atmosphere,customer service,temperature,price,speed,quality,courtesy
0,Why does it look like someone spit on my food I had a normal transaction everyone was chill and ...,"(hygiene, 0.28746825)","(food, 0.2742214)","(temperature of the food, 0.25352857)","(food quality, 0.19775563)","(courtesy, 0.1868012)",1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,Itd McDonalds It is what it is as far as the food and atmosphere go The staff here does make a d...,"(staff, 0.3886896)","(food quality, 0.36147058)","(customer service, 0.30244833)","(food, 0.29995057)","(temperature of the food, 0.22631292)",0,1,1,1,...,0,0,0,0,1,0,0,0,0,0
2,Made a mobile order got to the speaker and checked it in Line was not moving so I had to leave o...,"(customer service, 0.38279423)","(courtesy, 0.15833107)","(staff, 0.14755319)","(drive-thru, 0.14341448)","(something is missing, 0.14244273)",0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,My mc Crispy chicken sandwich was customer service was quick and p,"(food quality, 0.42417273)","(temperature of the food, 0.35612532)","(food, 0.33923125)","(customer service, 0.296566)","(quality, 0.2841521)",0,1,1,0,...,0,0,1,0,1,0,0,0,1,0
4,I repeat my order times in the drive thru and she still manage to mess it up it was suppose to b...,"(drive-thru, 0.40096334)","(courtesy, 0.2716957)","(food, 0.2629262)","(food quality, 0.22811925)","(customer service, 0.19910015)",0,0,1,0,...,0,1,0,0,0,0,0,0,0,1


In [ ]:
df[df["clean_reviews"]=="We love coming here Tried the Shamrock Mcflurry and it was delicious"][['clean_reviews', 'top_1_label', 'top_2_label', 'top_3_label', 'top_4_label',
       'top_5_label',  'hygiene', 'food quality', 'food',
       'staff', 'something is missing', 'location', 'speed of service',
       'drive-thru', 'temperature of the food', 'atmosphere',
       'customer service', 'temperature', 'price', 'speed', 'quality',
       'courtesy']]

,clean_reviews,top_1_label,top_2_label,top_3_label,top_4_label,top_5_label,hygiene,food quality,food,staff,...,speed of service,drive-thru,temperature of the food,atmosphere,customer service,temperature,price,speed,quality,courtesy
14073,We love coming here Tried the Shamrock Mcflurry and it was delicious,"(drive-thru, 0.2882666)","(food quality, 0.2093988)","(food, 0.1840965)","(temperature of the food, 0.14732102)","(location, 0.12049471)",0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
